In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\ML_Projects\\Kidney-Disease-Classification'

In [4]:
os.environ["MLFLOW_TRACKING_URL"] = "https://dagshub.com/ayushgandhi904/Kidney-Disease-Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "ayushgandhi904"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "3aaf0a139bb1f0dae8decd9e11e0bbe95af194b4"

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri:str
    params_image_size: list
    params_batch_size: int

In [8]:
from TumorClassifier.constants import *
from TumorClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConifgurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri = "https://dagshub.com/ayushgandhi904/Kidney-Disease-Classification.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        
        return eval_config

In [10]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [12]:
try:
    config = ConifgurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-11-30 22:28:47,548: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-30 22:28:47,548: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-30 22:28:47,556: INFO: common: created directory at: artifacts]
Found 60 images belonging to 2 classes.
4/4 [==============================] - 7s 2s/step - loss: 2.2132 - accuracy: 0.5333
[2023-11-30 22:28:55,460: INFO: common: json file saved at: scores.json]


2023/11/30 22:28:56 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-11-30 22:28:57,809: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\AYUSHG~1\AppData\Local\Temp\tmp_vanwny7\model\data\model\assets
[2023-11-30 22:28:58,745: INFO: builder_impl: Assets written to: C:\Users\AYUSHG~1\AppData\Local\Temp\tmp_vanwny7\model\data\model\assets]
[2023-11-30 22:29:07,035: INFO: dir_util: creating E:\ML_Projects\Kidney-Disease-Classification\mlruns\0\b0b85e6180974bc88fdccd2b46054ea6\artifacts\model\data]
[2023-11-30 22:29:07,042: INFO: dir_util: creating E:\ML_Projects\Kidney-Disease-Classification\mlruns\0\b0b85e6180974bc88fdccd2b46054ea6\artifacts\model\data\model]
[2023-11-30 22:29:07,042: INFO: dir_util: creating E:\ML_Projects\Kidney-Disease-Classification\mlruns\0\b0b85

e:\ML_Projects\Kidney-Disease-Classification\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
